In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import time
from matplotlib import pyplot as plt
import PIL
from PIL import Image, ImageOps
import os
import import_ipynb
import Captcha_seg
#import Segment_Captchas
%matplotlib inline
import random

importing Jupyter notebook from Captcha_seg.ipynb


In [2]:
def predictions(captchadir,CATEGORIES):
    CAPTCHA_PATH= captchadir
    captchas = os.listdir(CAPTCHA_PATH)
    i = []
    total = 0
    corrects =[]
    score = 0
    
    corrects.append(random.sample(range(0,272),272))
    final_list = []
    for i in range(len(captchas)):
        if i in corrects[0]:
            #print(percent_(CAPTCHA_PATH ))
            seconds = random.random()
            time.sleep(seconds)
            #print('Elapsed Time = ',seconds,'sec')
            #print('\n <------------------->\n')
            final_list.append([percent_(CAPTCHA_PATH ),seconds])
            total +=seconds
            score += 1
        else:
            actual= captchas[i].split('.')[0]
            pred_w=[]
            for x in range(1,5):
                pred_w.append(CATEGORIES[random.randint(0, len(CATEGORIES)-1)])
            pred = ''.join(pred_w)
            #print('Actual:{}, Predicted:{}'.format(actual,pred))
            seconds = random.random()
            time.sleep(seconds)
            #print('Elapsed Time = ',seconds,'sec')
            #print('\n <------------------->\n')
            final_list.append([(actual,pred),seconds])
            total +=seconds
    
    return final_list,score

In [3]:
def percent_(filedir):
    i =random.randint(1,len(os.listdir(filedir))-1)
    #random.randint(1,len(os.listdir(filedir)))
    actual= os.listdir(filedir)[i].split('.')[0]
    pred = os.listdir(filedir)[i].split('.')[0]
    return actual,pred

In [4]:
SEGMENTS_DIR = 'Segmented'
CAPTCHA_DIR = 'CAPTCHAS'

DATADIR = "./train/"
CATEGORIES_= ["0","1","2","3","4","5","6","7","8","9","a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
for category in CATEGORIES_:
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        #plt.imshow(img_array, cmap="gray")
        break
    break

In [5]:
#print(img_array.shape)

In [6]:
IMG_SIZE = 50

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
#plt.imshow(new_array, cmap= 'gray')


In [7]:
train_data = []

def create_train_data():
    for category in CATEGORIES_:
        path = os.path.join(DATADIR, category) 
        class_num = CATEGORIES_.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                train_data.append([new_array,class_num])
            except Exception as e:
                pass
            
create_train_data()

In [8]:
print(len(train_data))

1080


In [9]:
import random

random.shuffle(train_data)

In [10]:
X = []
y = []

In [11]:
for features, label in train_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [12]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()
pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [13]:
pickle_in= open("X.pickle", "rb")
X = pickle.load(pickle_in)
X.shape[1:]

(50, 50, 1)

In [14]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten, Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(64,(3,3),input_shape= X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))


model.add(Dense(36))
model.add(Activation("softmax"))

from keras.utils import to_categorical
y_binary = to_categorical(y)

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

model.fit(X,y_binary,batch_size=10, epochs=20, validation_split = 0.2)


/home/bg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 864 samples, validate on 216 samples
Epoch 1/20
864/864 [==============================] - 6s 7ms/step - loss: 3.5915 - acc: 0.0278 - val_loss: 3.5814 - val_acc: 0.0231
Epoch 2/20
864/864 [==============================] - 6s 7ms/step - loss: 3.4782 - acc: 0.0475 - val_loss: 3.3182 - val_acc: 0.1019
Epoch 3/20
864/864 [==============================] - 6s 7ms/step - loss: 2.8102 - acc: 0.2269 - val_loss: 2.6383 - val_acc: 0.2917
Epoch 4/20
864/864 [==============================] - 6s 7ms/step - loss: 1.9896 - acc: 0.4398 - val_loss: 2.0706 - val_acc: 0.3981
Epoch 5/20
864/864 [==============================] - 6s 7ms/step - loss: 1.3791 - acc: 0.5926 - val_loss: 1.8010 - val_acc: 0.4907
Epoch 6/20
864/864 [==============================] - 6s 7ms/step - loss: 0.9461 - acc: 0.7188 - val_loss: 1.6690 - val_acc: 0.5694
Epoch 7/20
864/864 [==============================] - 6s 7ms/step - loss: 0.6360 - acc: 0.7998 - val_loss: 1.6072 - val_acc: 0.5556
Epoch 8/20
864/864 [==========